In [1]:
%load_ext autoreload
%autoreload 2

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from hatedetection import HateSpeechAnalyzer
analyzer = HateSpeechAnalyzer.load_contextualized_model()

In [2]:

import ijson
from tqdm.auto import tqdm



path = "../../data/news-all-final.json"

num_articles = 537_200

data = []

num_comments = 0

LIMIT = 500
with open(path) as f:
    articles = ijson.items(f, 'item')

    for i, article in tqdm(enumerate(articles), total=num_articles):
        num_comments += len(article["comments"])

        context = article["title"] if "title" in article else article["text"]

        for comment in article["comments"]:
            data.append({
                "text": comment["text"],
                "context": context,
                "id": comment["tweet_id"],
            })
        
        if i > LIMIT:
            break

        

8.5M de comentarios. Ok, es un montonazo

In [3]:
import pandas as pd
from pandarallel import pandarallel
from hatedetection.preprocessing import preprocess_tweet

pandarallel.initialize()

df = pd.DataFrame(data)

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [4]:
device = "cuda"

model = analyzer.base_model
model = model.to(device)
model.eval()
tokenizer = analyzer.tokenizer

In [5]:
df["text"] = df["text"].parallel_apply(preprocess_tweet)
df["context"] = df["context"].parallel_apply(preprocess_tweet)

In [6]:
from hatedetection.training import tokenize
from datasets import Dataset, Value, ClassLabel, Features

features = Features({
    'id': Value('uint64'),
    'context': Value('string'),
    'text': Value('string'),
})

dataset = Dataset.from_pandas(df, features=features)

dataset = dataset.map(
    lambda x: tokenizer(x["text"], x["context"], padding="max_length", truncation=True), 
    batched=True,
)

def format_dataset(dataset):
    dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
    return dataset

dataset = format_dataset(dataset)

In [7]:
from torch.utils.data import DataLoader
from hatedetection import extended_hate_categories

dataloader = DataLoader(dataset, batch_size=16)
outputs = []

for batch in tqdm(dataloader):
    batch = {k:v.to(device) for k, v in batch.items() if k != "id"}
    outs = model(**batch)
    outputs.append((outs.logits > 0).cpu().numpy())


In [ ]:
import numpy as np

np.vstack(outputs)

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [ True, False, False, ..., False, False,  True],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [ ]:
import torch

